<a href="https://colab.research.google.com/github/tlcuzick/data-science-projects/blob/main/predicting-house-sale-prices/predicting_house_sale_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import KFold

# **Feature Engineering**

* Drop columns where more than 25% of values are null
* Drop all **text** columns with any null values
* Fill in missing numerical values with the overall means from their parent columns
* Add "years_sold" and "years_since_remod" features
* Drop rows with negative values for "years_sold" and "years_since_remod"
* Drop columns that either arent't useful for ML ("PID" for example) or leak info we wouldn't know about until the sale occurred (such as "Sale Condition")

In [2]:
def transform_features(data):
    num_missing = data.isnull().sum()
    max_missing = data.shape[0] / 4
    cols_to_drop = num_missing[num_missing > max_missing].index
    data = data.drop(cols_to_drop, axis=1)
    
    num_missing_txt = data.select_dtypes(include=['object']).isnull().sum()
    cols_to_drop_2 = num_missing_txt[num_missing_txt > 0].index
    data = data.drop(cols_to_drop_2, axis=1)
    
    num_missing_numeric = data.select_dtypes(include=['integer', 'float']).isnull().sum()
    cols_to_fix = num_missing_numeric[num_missing_numeric > 0].index
    means = data[cols_to_fix].mean()
    data = data.fillna(means)
    
    years_sold = data['Yr Sold'] - data['Year Built']
    years_since_remod = data['Yr Sold'] - data['Year Remod/Add']

    data['Years Before Sale'] = years_sold
    data['Years Since Remod'] = years_since_remod

    data = data.drop([2180, 1702, 2181], axis=0)
    data = data.drop(['Year Built', 'Year Remod/Add'], axis=1)

    data = data.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold", "PID", "Order"], axis=1)
    
    return data

# **Feature Selection**

* Remove columns with a correlation threshold less than 0.3
* Remove categorical features with more than 10 unique values
* Create "dummy" columns for categorical features with 10 or fewer unique values

In [3]:
def count_unique_vals(col):
    return len(col.value_counts())

def select_features(data, corr_threshold=0.3, uniq_threshold =10):
    numerical_transformed_features = data.select_dtypes(include=['integer','float'])
    
    sale_price_corr = numerical_transformed_features.corr()['SalePrice'].abs()
    
    cols_to_drop = sale_price_corr[sale_price_corr < corr_threshold].index
    
    data = data.drop(cols_to_drop, axis=1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                        "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                        "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                        "Misc Feature", "Sale Type", "Sale Condition"]
    nom_cols_to_transform = []

    for col in nominal_features:
        if col in data.columns:
            nom_cols_to_transform.append(col)
            
    unique_val_counts = data[nom_cols_to_transform].apply(count_unique_vals)
    nonunique_cols = unique_val_counts[unique_val_counts > 10].index
    
    data.drop(nonunique_cols, axis=1)
    
    text_cols = data.select_dtypes(include = ['object']).columns

    for col in text_cols:
        data[col] = data[col].astype('category')
        
    dummy_cols = pd.get_dummies(data.select_dtypes(include = ['category']))

    data = pd.concat([data, dummy_cols], axis=1)
    
    return data

# **Train and Test the Model**

In [4]:
def train_and_test(data, k=0):
    numeric_cols = data.select_dtypes(include=['integer', 'float'])
    features = numeric_cols.columns.drop('SalePrice')

    lr = linear_model.LinearRegression()
    
    if k == 0:
        train = data[:1460]
        test = data[1460:]
        
        lr.fit(train[features], train['SalePrice'])    
        predicted_sale_prices = lr.predict(test[features])

        mse = mean_squared_error(test['SalePrice'], predicted_sale_prices)
        rmse = mse ** (1/2)

        return rmse
    
    if k == 1:
        shuffled_data = data.sample(frac=1, )
        train = data[:1460]
        test = data[1460:]
        
        lr.fit(train[features], train["SalePrice"])
        predictions_one = lr.predict(test[features])        
        
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = mse_one ** (1/2)
        
        lr.fit(test[features], test["SalePrice"])
        predictions_two = lr.predict(train[features])        
       
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = mse_two ** (1/2)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print('RMSE one: {}'.format(rmse_one))
        print('RMSE two: {}'.format(rmse_two))
        return avg_rmse
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(data):
            train = data.iloc[train_index]
            test = data.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = mse  ** (1/2)
            rmse_values.append(rmse)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse

In [5]:
data = pd.read_csv('AmesHousing.tsv', delimiter="\t")
transformed_features = transform_features(data)
selected_features = select_features(transformed_features)
rmse = train_and_test(selected_features)
print(rmse)

32005.636765137042
